In [7]:
import pandas as pd 
from nltk.util import ngrams
from collections import Counter
import nltk 
nltk.download('stopwords') 
#import spacy

import pandas as pd
#from spacy.lang.tr import Turkish
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer



from nltk.corpus import stopwords 
stop_words = set(stopwords.words('turkish')) 
import nltk
nltk.download('wordnet')


data = pd.read_csv(r'train_3.csv',encoding='utf-8')

data.dropna(axis = 0 ,inplace=True)

data = data[['question_text']]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
import re
#import gensim
textList = []
for text in data.question_text:
    
    text = re.sub("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$",' ',text)
    text = re.sub(r'[^\w]', ' ', text)
    text = text.lower()
    text = nltk.word_tokenize(text) 
    text = [ word for word in text if not word in set(stopwords.words('turkish'))]

    text = ' '.join(text)
    textList.append(text)     
textFreq = pd.DataFrame(textList,columns = ['question_text'])

Frequences_ = textFreq.question_text.str.split(expand=True).stack().value_counts()

In [9]:
Freq = pd.DataFrame(Frequences_, columns = ['counts'])
Freq = Freq.reset_index()

In [18]:
most_freq = Freq[Freq['counts']>5].sort_values(by='counts',ascending=False)
least_freq = Freq[Freq['counts']<5].sort_values(by='counts',ascending=False)

In [20]:
print(most_freq[:50])

         index  counts
0          bir    2314
1       olarak     648
2         olan     396
3        sonra     269
4        kadar     269
5      türkiye     263
6        büyük     232
7            a     230
8          nin     224
9         yeni     224
10           2     205
11      ilgili     204
12  tarafından     194
13      içinde     185
14       genel     172
15       ancak     172
16           1     172
17        göre     170
18           e     170
19           3     169
20        türk     167
21         iki     167
22       madde     165
23       diğer     161
24      olduğu     160
25       zaman     147
26         yer     146
27        aynı     144
28          in     136
29       bilgi     133
30         ilk     132
31      önemli     131
32      yüksek     128
33          no     128
34         yıl     127
35         son     126
36      ayrıca     125
37        özel     122
38         iyi     120
39     şekilde     118
40    arasında     117
41    üzerinde     117
42        ö

In [21]:
print(least_freq[0:50])

                 index  counts
3895            ayları       4
4711          denizden       4
4718           genelge       4
4717         fransızca       4
4716               bit       4
4715            buğday       4
4714              uğur       4
4713            saglik       4
4712             adamı       4
4710            kıvanç       4
4739           yaninda       4
4709          hücreler       4
4708  gerçekleştirilen       4
4707             şayet       4
4706           yönerge       4
4705             meşru       4
4704           ödemesi       4
4703    etkilemektedir       4
4719           etkiler       4
4720      müsteşarlığı       4
4721              reis       4
4722              açar       4
4737        biliyordum       4
4736         şifrenizi       4
4735          düzenler       4
4734          sonuçsuz       4
4733   kaynaklanabilir       4
4732          tüketici       4
4731           turizmi       4
4730           saygılı       4
4729             onbeş       4
4728    

In [22]:
from distance import jaccard
column_names = ["error", "true", "similarity_distance"]
most_similar_ones = pd.DataFrame(columns = column_names)


def get_most_similars(dataframe):
    global most_similar_ones
    most_similar_ones= most_similar_ones.append(dataframe)

def jaccard_similarities():
    global column_names

    for y in least_freq['index'].values:
        error_word = []
        true_word = []
        similarities = []
        
        temporary_df = pd.DataFrame(columns = column_names)

        for x in most_freq['index'].values:
            error_word.append(y)
            true_word.append(x)
            similarities.append(1-jaccard(y, x))
        temporary_df['error'] = error_word
        temporary_df['true'] = true_word
        temporary_df['similarity_distance'] = similarities
        
        get_most_similars(temporary_df.sort_values(by='similarity_distance',ascending=False)[0:1])
        

jaccard_similarities()


In [23]:
most_similar_ones.sort_values(by='similarity_distance',ascending=False)[:50]

,error,true,similarity_distance
2586,ayları,yılları,1.0
348,öğrenir,örneğin,1.0
2930,kasko,sokak,1.0
136,çalışılmış,çalışma,1.0
2516,tartışmış,tartışma,1.0
310,karot,ortak,1.0
1457,1914,1994,1.0
1508,verem,verme,1.0
580,kasabası,baskı,1.0
1768,gerekti,gerektirir,1.0
